In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from scipy.stats import norm
from fredapi import Fred
import ipywidgets as widgets
from IPython.display import display, clear_output

# FRED API key
fred = Fred(api_key='YOUR_API_KEY')

# Function to get risk-free rate from FRED (using 3-month Treasury Bill as proxy)
def get_risk_free_rate():
    rate_data = fred.get_series('DTB3')  # 3-month Treasury Bill
    return rate_data.iloc[-1] / 100  # Get the latest rate

# Black-Scholes gamma calculation
def black_scholes_gamma(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    return gamma

# Function to fetch open interest and calculate net gamma exposure by strike
def calculate_net_gamma_and_open_interest(ticker_symbol, exp_date):
    ticker = yf.Ticker(ticker_symbol)
    underlying = ticker.history(period='1d')['Close'].iloc[-1]
    rf_rate = get_risk_free_rate()  # Get risk-free rate from FRED
    
    # Fetch options data for the expiration date
    opt_chain = ticker.option_chain(exp_date)
    calls = opt_chain.calls
    puts = opt_chain.puts

    # Store net gamma exposures and open interest
    gamma_exposures = {}
    open_interests = {'Calls': [], 'Puts': []}

    # Calculate Gamma for Calls and Puts and aggregate by strike
    for _, option in calls.iterrows():
        strike = option['strike']
        iv = option['impliedVolatility']
        T = (pd.to_datetime(exp_date) - pd.Timestamp.now()).days / 365  # Time to expiration

        if np.isnan(iv) or iv == 0:
            continue
        
        gamma = black_scholes_gamma(underlying, strike, rf_rate, iv, T)
        gamma_exposure = gamma * option['openInterest'] * 100  # For 100 shares per contract

        # Aggregate gamma for each strike
        if strike in gamma_exposures:
            gamma_exposures[strike] += gamma_exposure
        else:
            gamma_exposures[strike] = gamma_exposure
        
        # Collect open interest for calls
        open_interests['Calls'].append({'Strike': strike, 'Open Interest': option['openInterest']})
    
    for _, option in puts.iterrows():
        strike = option['strike']
        iv = option['impliedVolatility']
        T = (pd.to_datetime(exp_date) - pd.Timestamp.now()).days / 365  # Time to expiration
        
        if np.isnan(iv) or iv == 0:
            continue
        
        gamma = black_scholes_gamma(underlying, strike, rf_rate, iv, T)
        gamma_exposure = gamma * option['openInterest'] * 100  # For 100 shares per contract

        # Aggregate put gamma (subtracting from total)
        if strike in gamma_exposures:
            gamma_exposures[strike] -= gamma_exposure
        else:
            gamma_exposures[strike] = -gamma_exposure
        
        # Collect open interest for puts
        open_interests['Puts'].append({'Strike': strike, 'Open Interest': option['openInterest']})
    
    # Convert the dictionaries to DataFrames
    gamma_df = pd.DataFrame(list(gamma_exposures.items()), columns=['Strike', 'Net Gamma Exposure'])
    open_interest_calls_df = pd.DataFrame(open_interests['Calls'])
    open_interest_puts_df = pd.DataFrame(open_interests['Puts'])
    
    return gamma_df, open_interest_calls_df, open_interest_puts_df, underlying

# Function to plot both open interest and gamma exposure using Plotly, with vertical line for current price
def plot_open_interest_and_gamma(df_gamma, df_calls_oi, df_puts_oi, ticker_symbol, exp_date, underlying_price):
    # Plot Gamma Exposure with current price line
    fig_gamma = go.Figure()

    # Separate positive and negative net gamma exposures
    positive_gamma = df_gamma[df_gamma['Net Gamma Exposure'] > 0]
    negative_gamma = df_gamma[df_gamma['Net Gamma Exposure'] < 0]

    # Add bars for positive gamma (green) and negative gamma (red)
    fig_gamma.add_trace(go.Bar(
        x=positive_gamma['Strike'],
        y=positive_gamma['Net Gamma Exposure'],
        name='Positive Gamma Exposure',
        marker=dict(color='green')
    ))

    fig_gamma.add_trace(go.Bar(
        x=negative_gamma['Strike'],
        y=negative_gamma['Net Gamma Exposure'],
        name='Negative Gamma Exposure',
        marker=dict(color='red')
    ))

    # Add vertical dashed line for current price
    fig_gamma.add_vline(x=underlying_price, line=dict(color="yellow", dash="dash"), name="Current Price")
    fig_gamma.add_annotation(x=underlying_price, y=max(df_gamma['Net Gamma Exposure']),
                             text=f'Current Price: {underlying_price:.2f}', showarrow=True, arrowhead=2, ax=40)

    # Update layout for better visualization
    fig_gamma.update_layout(
        title=f'Net Gamma Exposure for {ticker_symbol.upper()} Expiring {exp_date}',
        xaxis_title='Strike Price',
        yaxis_title='Net Gamma Exposure',
        template='plotly_dark'
    )

    # Plot for Open Interest with current price line
    fig_oi = go.Figure()

    # Add bars for Calls Open Interest (blue)
    fig_oi.add_trace(go.Bar(
        x=df_calls_oi['Strike'],
        y=df_calls_oi['Open Interest'],
        name='Calls Open Interest',
        marker=dict(color='blue')
    ))

    # Add bars for Puts Open Interest (red)
    fig_oi.add_trace(go.Bar(
        x=df_puts_oi['Strike'],
        y=df_puts_oi['Open Interest'],
        name='Puts Open Interest',
        marker=dict(color='red')
    ))

    # Add vertical dashed line for current price
    fig_oi.add_vline(x=underlying_price, line=dict(color="yellow", dash="dash"), name="Current Price")
    fig_oi.add_annotation(x=underlying_price, y=max(max(df_calls_oi['Open Interest']), max(df_puts_oi['Open Interest'])),
                          text=f'Current Price: {underlying_price:.2f}', showarrow=True, arrowhead=2, ax=40)

    # Update layout for better visualization
    fig_oi.update_layout(
        title=f'Open Interest for {ticker_symbol.upper()} Expiring {exp_date}',
        xaxis_title='Strike Price',
        yaxis_title='Open Interest',
        template='plotly_dark'
    )

    # Show both plots
    fig_gamma.show()
    fig_oi.show()

# Widget for user input
ticker_input = widgets.Text(description="Ticker:")
expiration_dropdown = widgets.Dropdown(description="Expiration:", options=[])
generate_button = widgets.Button(description="Generate")

# Function to handle button click
def on_generate_button_clicked(b):
    clear_output()  # Clear previous output to replace old charts
    ticker = ticker_input.value
    try:
        expirations = yf.Ticker(ticker).options
        expiration_dropdown.options = expirations  # Update expiration options
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return
    
    # Display the updated UI
    display(ticker_input, expiration_dropdown, generate_button)

    def on_expiration_selected(change):
        expiration = expiration_dropdown.value
        gamma_df, calls_oi_df, puts_oi_df, underlying_price = calculate_net_gamma_and_open_interest(ticker, expiration)
        plot_open_interest_and_gamma(gamma_df, calls_oi_df, puts_oi_df, ticker, expiration, underlying_price)

    expiration_dropdown.observe(on_expiration_selected, names='value')

# Bind button click to the function
generate_button.on_click(on_generate_button_clicked)

# Display the widgets
display(ticker_input, expiration_dropdown, generate_button)